# Remaining issues:
## Handled issues:
- [solved] distance between geometries is in degrees, we need it in meters to determine sailing durations
- [solved] need to check resource requests closely to check why sharing ships isn't working yet
- [solved] need to add resource requests to sites
- [solved] need to add real time (plus a specific starting date)
- [solved] need to add logging to Sites to lists volumes at different timesteps (this helps to animate on Google Earth)

## Open issues:
- [open] need to add sensitivity to weather (see work Joris den Uijl)
- [open] need to add soil characteristics and turbidity generation (see work Joris den Uijl)
- [open] need to add routing via routing graph (Dijkstra algorithm)
- [open] need to make case handling web based (quick setup & quick case comparison)
- [open] need to collect the code in a package
- [open] change processing so that it can handle rate_in and rate_out in stead of just rate


# Create necessary classes

In [1]:
# package(s) related to time, space and id
import datetime
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import geojson
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
from  digital_twin.core import Identifiable, Site, TransportResource, TransportProcessingResource, ProcessingResource

## Activities

In [2]:
class Installation(Identifiable):
    """The Installation Class forms a spefic class of activities with associated methods that can 
    initiate and suspend processes according to a number of specified conditions. This class deals 
    with transport and installation/placement of discrete and continuous objects.
    
    condition: expression that states when to initiate or to suspend activity
    origin: object with simpy Container from which to get (can be Site or Vessel)
    destination: object with simpy Container in which to put (can be Site or Vessel)
    loader: gets amount from 'origin' Container and puts it into 'mover' Container
    mover: moves amount in Container from 'origin' to 'destination'
    unloader: gets amount from 'mover' Container and puts it into 'destination' Container"""

    def __init__(self, 
                 condition,
                 origin, destination,  
                 loader, mover, unloader,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        
        self.condition = condition
        self.origin = origin
        self.destination = destination
        self.loader = loader
        self.mover = mover
        self.unloader = unloader
        
        self.standing_by_proc = env.process(
            self.standing_by(env, 
                             condition,
                             origin, destination,
                             loader, mover, unloader))
        self.installation_proc = env.process(
            self.installation_process_control(env,
                             condition,
                             origin, destination,
                             loader, mover, unloader))
        self.installation_reactivate = env.event()

    def standing_by(self, env, condition,
                          origin, destination,
                          loader, mover, unloader):
        """Standing by"""
        shown = False

        while not eval(condition):
            if not shown:
                print('T=' + '{:06.2f}'.format(env.now) + ' ' + self.name + ' to ' + destination.name + ' suspended')
                shown = True
            yield env.timeout(3600) # step 3600 time units ahead

        print('T=' + '{:06.2f}'.format(env.now) + ' ' + 'Condition: ' + condition + ' is satisfied')

        self.installation_reactivate.succeed()  # "reactivate"
        self.installation_reactivate = env.event()
 
    def installation_process_control(self, env, condition,
                                           origin, destination,
                                           loader, mover, unloader):
        """Installation process control"""  
        while not eval(condition):
            yield self.installation_reactivate

        print('T=' + '{:06.2f}'.format(env.now) + ' '+ self.name + ' to ' + destination.name + ' started')
        while eval(condition):
            yield from self.installation_process(env, condition,
                                                 origin, destination,
                                                 loader, mover, unloader)

    def installation_process(self, env, condition,
                                   origin, destination,
                                   loader, mover, unloader):
        """Installation process"""
        # estimate amount that should be transported
        amount = min(
            mover.container.capacity - mover.container.level,
            origin.container.level,
            origin.container.capacity - origin.total_requested,
            destination.container.capacity - destination.container.level,
            destination.container.capacity - destination.total_requested)
        
        if amount>0:
            # request access to the transport_resource
            origin.total_requested += amount
            destination.total_requested += amount
            if id(loader) == id(mover): 
                # this is the case when a hopper is used
                with mover.resource.request() as my_mover_turn:
                    yield my_mover_turn

                    # request access to the load_resource
                    mover.log_entry('loading start', self.env.now, mover.container.level)
                    yield from loader.execute_process(origin, mover, amount)
                    mover.log_entry('loading stop', self.env.now, mover.container.level)

                    print('Loaded:')
                    print('  from:           ' + origin.name + ' contains: ' + str(origin.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + destination.name + ' contains: ' + str(destination.container.level))


                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.execute_move(origin, destination)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                    # request access to the placement_resource
                    mover.log_entry('unloading start', self.env.now, mover.container.level)
                    yield from unloader.execute_process(mover, destination, amount)
                    mover.log_entry('unloading stop', self.env.now, mover.container.level)

                    print('Unloaded:')
                    print('  from:           ' + destination.name + ' contains: ' + str(destination.container.level))
                    print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                    print('  to:             ' + origin.name + ' contains: ' + str(origin.container.level))

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.execute_move(destination, origin)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                    # once a mover is assigned to an Activity it completes a full cycle
                    mover.resource.release(my_mover_turn)
            else: 
                # if not a hopper is used we have to handle resource requests differently
                with mover.resource.request() as my_mover_turn:
                    yield my_mover_turn

                    # request access to the load_resource
                    with loader.resource.request() as my_load_resource_turn:
                        yield my_load_resource_turn

                        mover.log_entry('loading start', self.env.now, mover.container.level)
                        yield from loader.execute_process(origin, mover, amount)
                        mover.log_entry('loading stop', self.env.now, mover.container.level)

                        print('Loaded:')
                        print('  from:           ' + origin.name + ' contains: ' + str(origin.container.level))
                        print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                        print('  to:             ' + destination.name + ' contains: ' + str(destination.container.level))

                        loader.resource.release(my_load_resource_turn)

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.execute_move(origin, destination)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                    # request access to the placement_resource
                    with unloader.resource.request() as my_unloader_turn:
                        yield my_unloader_turn

                        mover.log_entry('unloading start', self.env.now, mover.container.level)
                        yield from unloader.execute_process(mover, destination, amount)
                        mover.log_entry('unloading stop', self.env.now, mover.container.level)

                        print('Unloaded:')
                        print('  from:           ' + destination.name + ' contains: ' + str(destination.container.level))
                        print('  by:             ' + mover.name + ' contains: ' + str(mover.container.level))
                        print('  to:             ' + origin.name + ' contains: ' + str(origin.container.level))

                        unloader.resource.release(my_unloader_turn)

                    mover.log_entry('sailing full start', self.env.now, mover.container.level)
                    yield from mover.execute_move(destination, origin)
                    mover.log_entry('sailing full stop', self.env.now, mover.container.level)

                    # once a mover is assigned to an Activity it completes a full cycle
                    mover.resource.release(my_mover_turn)
        else:
            yield env.timeout(3600)
            

# Start case

In [3]:
# *** Create a project environment
env = simpy.Environment()
start = env.now

In [4]:
# simulation returns time in seconds with epoch as the reference
env.epoch = datetime.datetime.now()

## Define sites

In [5]:
Sites = []
# *** Generate stock sites
# - sites in database
data_stock_01 = {"env": env,
                "name": "Stock 01", "geometry": geojson.Point([5.019298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_02 = {"env": env,
                "name": "Stock 02", "geometry": geojson.Point([5.271417603333632, 52.9638452897506]),
                "capacity": 150000, "level": 150000}
data_stock_03 = {"env": env,
                "name": "Stock 03", "geometry": geojson.Point([5.919298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}
data_stock_04 = {"env": env,
                "name": "Stock 04", "geometry": geojson.Point([5.919298185633251, 52.94239823421129]),
                "capacity": 150000, "level": 150000}

# - create site objects
stock_01 = Site(**data_stock_01)
Sites.append(stock_01)
stock_02 = Site(**data_stock_02)
Sites.append(stock_02)
stock_03 = Site(**data_stock_03)
Sites.append(stock_03)
stock_04 = Site(**data_stock_04)
Sites.append(stock_04)

# *** Generate placement sites
# - Clay layer
layer_name = '_clay'
capacity = 5000
level = 0
nums = 20
start = [5.054676856441372,52.94042293840172] # Den Oever 
stop = [5.294877712236641,53.06686424241725] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Sand layer
layer_name = '_sand'
capacity = 5000
level = 0
nums = 20
start = [5.052051052879287,52.9421894472733] # Den Oever 
stop = [5.292216781509101,53.06886359869087] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Armour layer
layer_name = '_armour'
capacity = 5000
level = 0
nums = 20
start = [5.049510554598302,52.94393628899332] # Den Oever 
stop = [5.289636346490858,53.07053144816584] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

# - Levvel layer
layer_name = '_levvel'
capacity = 5000
level = 0
nums = 20
start = [5.046556507026805,52.94579445406793] # Den Oever 
stop = [5.286775240694118,53.07264015015531] # Kornwerderzand

# - generate a 'nums' amount of sites between the selected start and stop points
lats = np.linspace(start[0], stop[0], num=nums)
lons = np.linspace(start[1], stop[1],  num=nums)

# - option to create a range of sites between two points
for i in range(nums):
    # - sites in database (nr indicates km's from Den Oever haven)
    data_site = {"env": env,
                "name": "KP" + format(i,'02.0f') + layer_name, "geometry": geojson.Point([lats[i], lons[i]]),
                "capacity": capacity, "level": level}
    
    # - create site objects
    vars()['Site_' + "KP" + format(i,'02.0f') + layer_name] = Site(**data_site)
    Sites.append(vars()['Site_' + "KP" + format(i,'02.0f') + layer_name])

## Define equipment

In [6]:
# *** Define fleet

# sites in database (nr indicates km's from Den Oever haven)
# - processing resources
data_gantry_crane = {"env": env,
                "name": "Gantry crane", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "rate": 0.10, "nr_resources": 1}
data_installation_crane = {"env": env,
                "name": "Installation crane", "geometry": geojson.Point([53.0229621352376,  5.197016484858931]),
                "rate": 0.05, "nr_resources": 1}

# - transport resources
data_transport_barge_01 = {"env": env,
                "name": "Transport barge 01", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "v_empty":1.6, "v_full":1}
data_transport_barge_02 = {"env": env,
                "name": "Transport barge 02", "geometry": geojson.Point([52.93917167503315, 5.070195628786471]),
                "capacity": 1000, "level": 0, "nr_resources": 1, "v_empty":1.6, "v_full":1}

# - transport processing resources
data_hopper = {"env": env,
                "name": "Hopper", "geometry": geojson.Point([52.94239823421129, 5.019298185633251]),
                "rate": 2, "nr_resources": 1, "capacity": 1000, "level": 0,  "nr_resources": 1, "v_empty":2, "v_full":1.5}

# create site objects
# - processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# - transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# - transport processing resources
hopper = TransportProcessingResource(**data_hopper)

## Define activities

In [7]:
# *** Define installation activities
transport_barges=[]
transport_barges.append(transport_barge_01)
transport_barges.append(transport_barge_02)

# Clay
layer_name = '_clay'
for i in range(nums):
    for transport_barge in transport_barges:
        # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition = "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''"
            data_act = {"env": env,
                    "name": "Clay placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                        ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + "'''"
            data_act = {"env": env,
                    "name": "Clay placement",
                    "origin": stock_01, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}

        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Sand
layer_name = '_sand'
previous_layer_name = '_clay'
for i in range(nums):
    # - sites in database (nr i indicates km's from Den Oever haven)
    if i==0:
        condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
        data_act = {"env": env,
                "name": "Sand placement",
                "origin": stock_02, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                "loader": hopper, "mover": hopper, "unloader": hopper,
                "condition": eval(condition)}
    else:
        condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
        data_act = {"env": env,
                "name": "Sand placement",
                "origin": stock_02, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                "loader": hopper, "mover": hopper, "unloader": hopper,
                "condition": eval(condition)}

    # - create site objects
    vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Armour
layer_name = '_armour'
previous_layer_name = '_sand'
for i in range(nums):
    for transport_barge in transport_barges:
        # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                        ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": env,
                    "name": "Armour placement",
                    "origin": stock_03, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": env,
                    "name": "Armour placement",
                    "origin": stock_03, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}

        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)

# Levvel
layer_name = '_levvel'
previous_layer_name = '_armour'
for i in range(nums):
    for transport_barge in transport_barges:
    # - sites in database (nr i indicates km's from Den Oever haven)
        if i==0:
            condition =  "'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000" + \
                        ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": env,
                    "name": "Block placement",
                    "origin": stock_04, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
        else:
            condition = "'''" + eval("'''Site_KP" + format(i,'02.0f') + layer_name + ".container.level<5000'''") + \
                    ' and ' + eval("'''Site_KP" + format(i-1,'02.0f') + layer_name + ".container.level==5000'''") + \
                    ' and ' + "Site_KP" + format(i,'02.0f') + previous_layer_name + ".container.level==5000'''"
            data_act = {"env": env,
                    "name": "Block placement",
                    "origin": stock_04, "destination": vars()['Site_' + "KP" + format(i,'02.0f') + layer_name],
                    "loader": gantry_crane, "mover": transport_barge, "unloader": installation_crane,
                    "condition": eval(condition)}
    
        # - create site objects
        vars()['Act_' + format(i,'02.0f') + layer_name] = Installation(**data_act)
            

## Run simulation

In [8]:
#*** Run the project
env.run()

T=000.00 Condition: Site_KP00_clay.container.level<5000 is satisfied
T=000.00 Clay placement to KP00_clay started
T=000.00 Condition: Site_KP00_clay.container.level<5000 is satisfied
T=000.00 Clay placement to KP00_clay started
T=000.00 Clay placement to KP01_clay suspended
T=000.00 Clay placement to KP01_clay suspended
T=000.00 Clay placement to KP02_clay suspended
T=000.00 Clay placement to KP02_clay suspended
T=000.00 Clay placement to KP03_clay suspended
T=000.00 Clay placement to KP03_clay suspended
T=000.00 Clay placement to KP04_clay suspended
T=000.00 Clay placement to KP04_clay suspended
T=000.00 Clay placement to KP05_clay suspended
T=000.00 Clay placement to KP05_clay suspended
T=000.00 Clay placement to KP06_clay suspended
T=000.00 Clay placement to KP06_clay suspended
T=000.00 Clay placement to KP07_clay suspended
T=000.00 Clay placement to KP07_clay suspended
T=000.00 Clay placement to KP08_clay suspended
T=000.00 Clay placement to KP08_clay suspended
T=000.00 Clay placem

  distance empty: 14939.31 m
  sailing empty:  2.00 m/s
  duration:       2.07 hrs
  distance full:  3269.83 m
  sailing full:   1.00 m/s
  duration:       0.91 hrs
  process:        0.14 hrs
Loaded:
  from:           Stock 02 contains: 145000
  by:             Hopper contains: 1000
  to:             KP00_sand contains: 4000
  process:        5.56 hrs
Unloaded:
  from:           KP01_clay contains: 3000
  by:             Transport barge 02 contains: 0
  to:             Stock 01 contains: 141000
  distance empty: 3269.83 m
  sailing empty:  1.60 m/s
  duration:       0.57 hrs
  distance full:  14939.31 m
  sailing full:   1.50 m/s
  duration:       2.77 hrs
  process:        0.14 hrs
Unloaded:
  from:           KP00_sand contains: 5000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 145000
T=180000.00 Condition: Site_KP00_armour.container.level<5000 and Site_KP00_sand.container.level==5000 is satisfied
T=180000.00 Condition: Site_KP00_armour.container.level<500

  distance empty: 13992.79 m
  sailing empty:  2.00 m/s
  duration:       1.94 hrs
  distance full:  58471.67 m
  sailing full:   1.00 m/s
  duration:       16.24 hrs
  process:        5.56 hrs
Unloaded:
  from:           KP00_armour contains: 1000
  by:             Transport barge 01 contains: 0
  to:             Stock 03 contains: 148000
  process:        5.56 hrs
Unloaded:
  from:           KP00_armour contains: 2000
  by:             Transport barge 02 contains: 0
  to:             Stock 03 contains: 148000
  distance empty: 58471.67 m
  sailing empty:  1.60 m/s
  duration:       10.15 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 01 contains: 139000
  by:             Transport barge 01 contains: 1000
  to:             KP02_clay contains: 0
  distance full:  4268.26 m
  sailing full:   1.00 m/s
  duration:       1.19 hrs
  distance empty: 58471.67 m
  sailing empty:  1.60 m/s
  duration:       10.15 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 01 

  process:        5.56 hrs
Unloaded:
  from:           KP00_armour contains: 4000
  by:             Transport barge 02 contains: 0
  to:             Stock 03 contains: 146000
  distance empty: 58471.67 m
  sailing empty:  1.60 m/s
  duration:       10.15 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 01 contains: 137000
  by:             Transport barge 01 contains: 1000
  to:             KP02_clay contains: 2000
  distance full:  4268.26 m
  sailing full:   1.00 m/s
  duration:       1.19 hrs
  distance empty: 58471.67 m
  sailing empty:  1.60 m/s
  duration:       10.15 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 01 contains: 136000
  by:             Transport barge 02 contains: 1000
  to:             KP02_clay contains: 3000
  process:        5.56 hrs
Unloaded:
  from:           KP02_clay contains: 3000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 136000
  distance full:  4268.26 m
  sailing full:   1.00 m/s

  distance full:  13075.36 m
  sailing full:   1.50 m/s
  duration:       2.42 hrs
  process:        0.14 hrs
Unloaded:
  from:           KP02_sand contains: 1000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 139000
  distance empty: 13075.36 m
  sailing empty:  2.00 m/s
  duration:       1.82 hrs
  process:        0.14 hrs
Loaded:
  from:           Stock 02 contains: 138000
  by:             Hopper contains: 1000
  to:             KP02_sand contains: 1000
  process:        5.56 hrs
Unloaded:
  from:           KP02_clay contains: 5000
  by:             Transport barge 01 contains: 0
  to:             Stock 01 contains: 135000
  distance empty: 4268.26 m
  sailing empty:  1.60 m/s
  duration:       0.74 hrs
  distance full:  13075.36 m
  sailing full:   1.50 m/s
  duration:       2.42 hrs
  process:        0.14 hrs
Unloaded:
  from:           KP02_sand contains: 2000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 138000
  process:  

...

  distance empty: 17503.96 m
  sailing empty:  1.60 m/s
  duration:       3.04 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 04 contains: 93000
  by:             Transport barge 01 contains: 1000
  to:             KP11_levvel contains: 0
  distance full:  50014.32 m
  sailing full:   1.00 m/s
  duration:       13.89 hrs
  distance full:  50014.32 m
  sailing full:   1.00 m/s
  duration:       13.89 hrs
  process:        5.56 hrs
Unloaded:
  from:           KP11_levvel contains: 1000
  by:             Transport barge 02 contains: 0
  to:             Stock 04 contains: 93000
  process:        5.56 hrs
Unloaded:
  from:           KP11_levvel contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 93000
  distance empty: 50014.32 m
  sailing empty:  1.60 m/s
  duration:       8.68 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 03 contains: 85000
  by:             Transport barge 02 contains: 1000
  to:           


Loaded:
  from:           Stock 03 contains: 81000
  by:             Transport barge 02 contains: 1000
  to:             KP13_armour contains: 3000
  distance full:  49308.78 m
  sailing full:   1.00 m/s
  duration:       13.70 hrs
  process:        5.56 hrs
Unloaded:
  from:           KP12_levvel contains: 2000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 88000
  distance full:  48384.33 m
  sailing full:   1.00 m/s
  duration:       13.44 hrs
  distance empty: 49308.78 m
  sailing empty:  1.60 m/s
  duration:       8.56 hrs
  process:        5.56 hrs
Unloaded:
  from:           KP13_armour contains: 4000
  by:             Transport barge 02 contains: 0
  to:             Stock 03 contains: 81000
  process:        2.78 hrs
Loaded:
  from:           Stock 01 contains: 72000
  by:             Transport barge 01 contains: 1000
  to:             KP15_clay contains: 2000
  distance full:  18624.46 m
  sailing full:   1.00 m/s
  duration:       5.17 

  process:        2.78 hrs
Loaded:
  from:           Stock 03 contains: 75000
  by:             Transport barge 02 contains: 1000
  to:             KP14_armour contains: 4000
  distance empty: 19745.46 m
  sailing empty:  1.60 m/s
  duration:       3.43 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 04 contains: 82000
  by:             Transport barge 01 contains: 1000
  to:             KP13_levvel contains: 2000
  distance full:  47708.52 m
  sailing full:   1.00 m/s
  duration:       13.25 hrs
T=10551600.00 Condition: Site_KP15_armour.container.level<5000 and Site_KP14_armour.container.level==5000 and Site_KP15_sand.container.level==5000 is satisfied
T=10551600.00 Condition: Site_KP15_armour.container.level<5000 and Site_KP14_armour.container.level==5000 and Site_KP15_sand.container.level==5000 is satisfied
T=10551600.00 Armour placement to KP15_armour started
T=10551600.00 Armour placement to KP15_armour started
  process:        5.56 hrs
Unloaded:
  from:           

Loaded:
  from:           Stock 01 contains: 59000
  by:             Transport barge 01 contains: 1000
  to:             KP18_clay contains: 0
  distance full:  10207.64 m
  sailing full:   1.50 m/s
  duration:       1.89 hrs
  process:        0.14 hrs
Unloaded:
  from:           KP17_sand contains: 3000
  by:             Hopper contains: 0
  to:             Stock 02 contains: 62000
  distance empty: 10207.64 m
  sailing empty:  2.00 m/s
  duration:       1.42 hrs
  process:        0.14 hrs
Loaded:
  from:           Stock 02 contains: 61000
  by:             Hopper contains: 1000
  to:             KP17_sand contains: 3000
  process:        2.78 hrs
Loaded:
  from:           Stock 04 contains: 75000
  by:             Transport barge 02 contains: 1000
  to:             KP14_levvel contains: 4000
  distance full:  10207.64 m
  sailing full:   1.50 m/s
  duration:       1.89 hrs
  process:        0.14 hrs
Unloaded:
  from:           KP17_sand contains: 4000
  by:             Hopper contain


Unloaded:
  from:           KP16_levvel contains: 4000
  by:             Transport barge 01 contains: 0
  to:             Stock 04 contains: 66000
  distance empty: 45188.25 m
  sailing empty:  1.60 m/s
  duration:       7.85 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 04 contains: 65000
  by:             Transport barge 02 contains: 1000
  to:             KP16_levvel contains: 4000
  distance empty: 46655.75 m
  sailing empty:  1.60 m/s
  duration:       8.10 hrs
  process:        2.78 hrs
Loaded:
  from:           Stock 03 contains: 58000
  by:             Transport barge 01 contains: 1000
  to:             KP18_armour contains: 1000
  distance full:  46655.75 m
  sailing full:   1.00 m/s
  duration:       12.96 hrs
T=12895200.00 Condition: Site_KP17_levvel.container.level<5000 and Site_KP16_levvel.container.level==5000 and Site_KP17_armour.container.level==5000 is satisfied
T=12895200.00 Condition: Site_KP17_levvel.container.level<5000 and Site_KP16_levvel.contai

## Some basic visualisation on Google Earth

In [11]:
icon = 'http://maps.google.com/mapfiles/kml/shapes/donut.png'
size = 1

kml = Kml()
fol = kml.newfolder(name="A Folder")

shared_style = Style()
shared_style.labelstyle.color = 'ffffffff'  # White
shared_style.labelstyle.scale = 1  
shared_style.iconstyle.color = 'ffff0000'  # Blue
shared_style.iconstyle.scale = 1
shared_style.iconstyle.icon.href = icon

for site in Sites:
    if not site.value or len(site.value) < 2:
        pnt = fol.newpoint(name=site.name, coords=[site.geometry["coordinates"]])
        pnt.timestamp.when = env.epoch.isoformat()
        pnt.style = shared_style
    else:
        # ignore last point because we need an endpoint
        for i, value in enumerate(site.value[:-1]):
            # convert to real dates
            begin = env.epoch + datetime.timedelta(seconds=site.t[i])
            end = env.epoch + datetime.timedelta(seconds=site.t[i+1])
            pnt = fol.newpoint(name='', coords=[site.geometry["coordinates"]])           
            # convert to string
            pnt.timespan.begin = begin.isoformat()
            pnt.timespan.end = end.isoformat()
            # use custom style if we are time dependent
            style = Style()
            style.labelstyle.color = 'ffffffff'  # White
            style.labelstyle.scale = 1  
            style.iconstyle.color = 'ffff0000'  # Blue
            style.iconstyle.scale = (value / site.container.capacity) * size
            style.iconstyle.icon.href = icon
            pnt.style = style
        begin = env.epoch + datetime.timedelta(seconds=site.t[-1])
        end = env.epoch + datetime.timedelta(seconds=env.now)
        pnt = fol.newpoint(name='', coords=[site.geometry["coordinates"]])           
        # convert to string
        pnt.timespan.begin = begin.isoformat()
        pnt.timespan.end = end.isoformat()
        # use custom style if we are time dependent
        style = Style()
        style.labelstyle.color = 'ffffffff'  # White
        style.labelstyle.scale = 1  
        style.iconstyle.color = 'ff00ff00'   # Green
        style.iconstyle.scale = (site.value[-1] / site.container.capacity) * size
        style.iconstyle.icon.href = icon
        pnt.style = style

kml.save("sharedstyle.kml")

In [12]:
# open the file
if platform.system():
    !start ./sharedstyle.kml
else:
    !start explorer ./sharedstyle.kml